# Probabilistic Programming Module


Here, we discuss how to use the probabilistic programming module to synthetically generate CDMs.



In [2]:
import kessler
import pyprob
import numpy as np
import dsgp4

In [3]:
#we seed everything for reproducibility
pyprob.seed(1)

#we define the observing instruments
#GNSS first:
gnss_cov_rtn=np.array([1e-9, 1.115849341564346, 0.059309835843067, 1e-9, 1e-9, 1e-9])**2,
instrument_characteristics_gnss={'bias_xyz': np.array([[0., 0., 0.],
       [0., 0., 0.]]), 'covariance_rtn': gnss_cov_rtn}
gnss = kessler.GNSS(instrument_characteristics_gnss)

#and then radar:
radar_cov_rtn=np.array([1.9628939405514678, 2.2307686944695706, 0.9660907831563862, 1e-9, 1e-9, 1e-9])**2
instrument_characteristics_radar={'bias_xyz': np.array([[0., 0., 0.],
       [0., 0., 0.]]), 'covariance_rtn': radar_cov_rtn}
radar = kessler.Radar(instrument_characteristics_radar)


In [4]:
tles=dsgp4.tle.load('tles_sample_population.txt')
print(f"loaded {len(tles)} TLEs")

loaded 20 TLEs


In [6]:
# tles=dsgp4.tle.load('tles_sample_population.txt')

# tles_filtered=[]
# for tle in tqdm(tles):
#     try: 
#         dsgp4.initialize_tle(tle)
#         if tle.apogee_alt()<560*1e3:
#             tles_filtered.append(tle)
#     except:
#         continue

In [5]:
conjunction_model = kessler.model.ConjunctionSimplified(time0=60727.13899462018,
                                                        # max_duration_days=7.0,
                                                        # time_resolution=600000.0,
                                                        # time_upsample_factor=100,
                                                        miss_dist_threshold=5000.0,
                                                        prior_dict=None,
                                                        t_prob_new_obs=0.96,
                                                        c_prob_new_obs=0.4,
                                                        cdm_update_every_hours=8.0,
                                                        mc_samples=100,
                                                        mc_upsample_factor=100,
                                                        pc_method='MC',
                                                        collision_threshold=70,
                                                        # likelihood_t_stddev=[371.068006, 0.0999999999, 0.172560879],
                                                        # likelihood_c_stddev=[371.068006, 0.0999999999, 0.172560879],
                                                        likelihood_time_to_tca_stddev=0.7,
                                                        t_observing_instruments=[gnss],
                                                        c_observing_instruments=[radar],
                                                        tles=tles,)

In [34]:
trace=conjunction_model.get_conjunction()

After 835 iterations, generated event with 7 CDMs


In [25]:
trace[0]['cdms']

[CCSDS_CDM_VERS                        = 1.0
 CREATION_DATE                         = 2025-02-21T03:20:09.135184
 ORIGINATOR                            = KESSLER_SOFTWARE
 MESSAGE_ID                            = KESSLER_SOFTWARE_65189812-1387-11f0-a1a9-f2a9f71f7e19
 TCA                                   = 2025-02-23T13:01:31.347952
 MISS_DISTANCE                         = 620.5391897367289
 RELATIVE_SPEED                        = 8.500887282727685
 RELATIVE_POSITION_R                   = -30.90865879375995
 RELATIVE_POSITION_T                   = -580.2512313942899
 RELATIVE_POSITION_N                   = -217.76604252292663
 RELATIVE_VELOCITY_R                   = -0.1061598754408795
 RELATIVE_VELOCITY_T                   = -5.096198697553176
 RELATIVE_VELOCITY_N                   = -6.803129684898339
 COLLISION_PROBABILITY                 = 0.0
 COLLISION_PROBABILITY_METHOD          = MC
 OBJECT                                = OBJECT1
 OBJECT_DESIGNATOR                     = 76126AP

In [35]:
#let's save all cdms to kvn files:
#for i in range(len(trace[0]['cdms'])):    
#    trace[0]['cdms'][i].save(f'event2_{i}.kvn')